<a href="https://colab.research.google.com/github/sarilacivert/3/blob/main/CLtest01e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Handwritten Digit Recognition**

In [1]:
%pip install git+https://github.com/PyTorchLightning/pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-ole1tli6
  Running command git clone -q https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-ole1tli6
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 140 kB 14.2 MB/s 
     |████████████████████████████████| 419 kB 66.8 MB/s 
     |████████████████████████████████| 596 kB 64.6 MB/s 
     |████████████████████████████████| 1.1 MB 56.8 MB/s 
     |████████████████████████████████| 94 kB 4.6 MB/s 
     |████████████████████████████████| 144 kB 75.8 MB/s 
     |████████████████████████████████| 271 kB 76.6 MB/s 
  Created wheel for pytorch-lightning: filename=pytorch_lightning-1.7.0.dev0-py3-none-any.whl size=607

In [2]:
%pip install git+https://github.com/PyTorchLightning/lightning-bolts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/PyTorchLightning/lightning-bolts to /tmp/pip-req-build-nonj_so4
  Running command git clone -q https://github.com/PyTorchLightning/lightning-bolts /tmp/pip-req-build-nonj_so4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for lightning-bolts: filename=lightning_bolts-0.6.0.dev0-py3-none-any.whl size=323226 sha256=beb2f00f442fb874b7e61899359a6ea224d16ddb55a7d534f9237d97596f72fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-25i46brh/wheels/5e/44/7e/f226440c020d6e3c2ba3d24fef794a29104e8ceb8b2d006058
Successfully built lightning-bolts


In [3]:
!pip freeze | grep torch

pytorch-lightning @ git+https://github.com/PyTorchLightning/pytorch-lightning@15b64cf8bac457e47a19102dd8152ddfb1821b72
torch @ https://download.pytorch.org/whl/cu113/torch-1.11.0%2Bcu113-cp37-cp37m-linux_x86_64.whl
torchaudio @ https://download.pytorch.org/whl/cu113/torchaudio-0.11.0%2Bcu113-cp37-cp37m-linux_x86_64.whl
torchmetrics==0.9.1
torchsummary==1.5.1
torchtext==0.12.0
torchvision @ https://download.pytorch.org/whl/cu113/torchvision-0.12.0%2Bcu113-cp37-cp37m-linux_x86_64.whl


In [4]:
import torch

In [5]:
import pytorch_lightning as pl

In [6]:
from torchvision.models import resnet18
from torch import nn
from torch.utils.data import DataLoader

In [7]:
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

In [8]:
train_ds = MNIST("mnist", train=True, download=True, transform=ToTensor())
test_ds = MNIST("mnist", train=False, download=True, transform=ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting mnist/MNIST/raw/train-images-idx3-ubyte.gz to mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting mnist/MNIST/raw/train-labels-idx1-ubyte.gz to mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist/MNIST/raw



In [9]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=64)

In [10]:
class ResNetMNIST(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = resnet18(num_classes=10)
    self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    self.loss = nn.CrossEntropyLoss()

  def forward(self, x):
    return self.model(x)
  
  def training_step(self, batch, batch_no):
    x, y = batch
    logits = self(x)
    loss = self.loss(logits, y)
    return loss
  
  def configure_optimizers(self):
    return torch.optim.RMSprop(self.parameters(), lr=0.005)

In [11]:
model = ResNetMNIST()

In [12]:
trainer = pl.Trainer(
    gpus=1,
    max_epochs=1
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:431: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  f"Setting `Trainer(gpus={gpus!r})` is deprecated in v1.7 and will be removed"
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(model, train_dl)

Missing logger folder: /content/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type             | Params
-------------------------------------------
0 | model | ResNet           | 11.2 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.701    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [14]:
trainer.save_checkpoint("resnet18_mnist.pt")

In [15]:
def get_prediction(x, model: pl.LightningModule):
  model.freeze() # prepares model for predicting
  probabilities = torch.softmax(model(x), dim=1)
  predicted_class = torch.argmax(probabilities, dim=1)
  return predicted_class, probabilities

In [16]:
from tqdm.autonotebook import tqdm

In [17]:
inference_model = ResNetMNIST.load_from_checkpoint("resnet18_mnist.pt", map_location="cuda")

In [18]:
true_y, pred_y = [], []
for batch in tqdm(iter(test_dl), total=len(test_dl)):
  x, y = batch
  true_y.extend(y)
  preds, probs = get_prediction(x, inference_model)
  pred_y.extend(preds.cpu())

  0%|          | 0/157 [00:00<?, ?it/s]

In [19]:
from sklearn.metrics import classification_report

In [20]:
print(classification_report(true_y, pred_y, digits=3))

              precision    recall  f1-score   support

           0      0.977     0.996     0.986       980
           1      0.977     0.994     0.986      1135
           2      0.985     0.977     0.981      1032
           3      0.996     0.973     0.984      1010
           4      0.994     0.941     0.967       982
           5      0.980     0.985     0.983       892
           6      0.973     0.987     0.980       958
           7      0.930     0.988     0.958      1028
           8      0.992     0.965     0.978       974
           9      0.965     0.953     0.959      1009

    accuracy                          0.976     10000
   macro avg      0.977     0.976     0.976     10000
weighted avg      0.977     0.976     0.976     10000

